In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D
import numpy as np
import os
from keras.optimizers import *
from keras.utils.generic_utils import Progbar
import matplotlib.pyplot as plt

RUN = 'C'
OUT_DIR = 'out/' + RUN
TENSORBOARD_DIR = 'tensorboard/wgans/' + RUN
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)
if not os.path.isdir(TENSORBOARD_DIR): os.makedirs(TENSORBOARD_DIR)
    
def resize_with_resolution(resize,img):
    x = resize[1]
    y = resize[0]
    width = len(img[1])
    height = len(img[0])
    img2 = np.zeros((width*x,height*y))
    for i in range(width):
        for j in range(height):
            img2[x*i:x*(i+1),y*j:y*(j+1)] = img[i,j]
    return img2


sample = 50
data_input = np.zeros((sample,10,10,1))
data_output = np.zeros((sample,10,10,1))
for k in range(sample):
    i = np.random.randint(0,9)
    j = np.random.randint(0,6)
    data_input[k,i:i+2,j:j+2,:] = 1
    data_output[k,i:i+2, j+3:j+5,:] = 1
    
test = np.zeros((1,10,10,1))
test[0,2:4,2:4,0] = 1

epoch = 1000
batch_size = 3

model = Sequential([
    Conv2D(1,(7,7), input_shape=(10,10,1),kernel_initializer='he_uniform', padding='same'),
    #Conv2D(1,(1,1), input_shape=(10,10,1),kernel_initializer='he_uniform', padding='same'),
])

model.compile(optimizer=RMSprop(lr=0.001),loss='mse')

progress_bar = Progbar(target=epoch)

for i in range(epoch):
    index = np.random.choice(sample, batch_size, replace=False)
    batch_input = data_input[index]
    batch_output = data_output[index]
    model.train_on_batch(batch_input, batch_output)
    if i%100==0:
        progress_bar.update(i)
        out = resize_with_resolution((20,20),model.predict(test)[0,:,:,0])
        plt.imsave(OUT_DIR + '/samples_%07d.png' % i, out, cmap=plt.cm.gray)


 900/1000 [==========================>...] - ETA: 13s